## Optimize codes in Python

#### Mingzhang Yang
#### 8-18-2016

1. Get it right
2. Test it's right
3. Profile if slow
4. Optimize
5. Repeat from 2

Ref: https://wiki.python.org/moin/PythonSpeed/PerformanceTips

--------
__Assignment__

Given a pool, namely a list of 1000 numbers. Randomly pick up 30 from the pool. Calculate the mean of the sample. Repeat it 10000 times to get the the collection of sample means.

In [1]:
aList = list(range(1000))
print(type(aList))

<class 'list'>


### Solution #1. Pure Python

__1.1__ For loop + list.append()

In [2]:
import random
import time

In [3]:
res = []

In [4]:
start = time.time()
for i in range(10000):
    temp = random.sample(aList, 30)
    res.append(sum(temp)/30)
    
print("Time used: %f s" % (time.time() - start))

Time used: 0.489275 s


In [5]:
len(res)

10000

In [6]:
def myfunc(n):
    res = []
    for i in range(n):
        temp = random.sample(aList, 30)
        res.append(sum(temp)/30)
    return res

%timeit(myfunc(10000))

1 loop, best of 3: 471 ms per loop


In [7]:
def myfunc(n):
    res = []
    for i in range(n):
        temp = random.sample(aList, 30)
        res.insert(0, sum(temp)/30)
    return res

%timeit(myfunc(10000))

1 loop, best of 3: 516 ms per loop


__1.2__ For loop + dict

In [8]:
res = {}

In [9]:
start = time.time()
for i in range(10000):
    temp = random.sample(aList, 30)
    res[i] = sum(temp) / 30
    
print("Time used: %f s" % (time.time() - start))

Time used: 0.507463 s


__1.3__ For loop + array

In [10]:
import array

In [11]:
res = array.array("f", [])

In [12]:
start = time.time()
for i in range(10000):
    temp = random.sample(aList, 30)
    res.append(sum(temp) / 30)
    
print("Time used: %f s" % (time.time() - start))

Time used: 0.407046 s


__1.4__ List comprehension

In [13]:
start = time.time()

res = [(sum(random.sample(aList, 30)) / 30) for i in range(10000)]

print("Time used: %f s" % (time.time() - start))

Time used: 0.532696 s


In [14]:
def sample_mean(seq, n):
    return sum(random.sample(seq, n)) / n

In [15]:
start = time.time()

res = [sample_mean(aList, 30) for i in range(10000)]

print("Time used: %f s" % (time.time() - start))

Time used: 0.455440 s


### Solution #2. Numpy

In [16]:
import numpy as np

__2.1__ For loop + array extension

In [17]:
res = np.array([])

In [18]:
start = time.time()

for i in range(10000):
    temp = np.random.choice(1000, 30).mean()
    res = np.append(res, temp)
    
print("Time used: %f s" % (time.time() - start))

Time used: 0.748540 s


__2.2__ For loop + assignment

In [19]:
res = np.empty(10000)

In [20]:
start = time.time()

for i in range(10000):
    temp = np.random.choice(1000, 30).mean()
    res[i] = temp
    
print("Time used: %f s" % (time.time() - start))

Time used: 0.373080 s


__2.3__ For loop + ndarray

In [21]:
res = np.empty((10000, 30))

In [22]:
start = time.time()

for i in range(10000):
    res[i, :] = np.random.choice(1000, 30)
    
res = res.mean(axis = 1)

print("Time used: %f s" % (time.time() - start))

Time used: 0.258628 s


In [23]:
print(res.shape)

(10000,)


__2.4__ np.apply_along_axis

In [24]:
start = time.time()

pool = np.repeat(np.arange(1000).reshape(1, 1000), 10000, axis=0)

res = np.apply_along_axis(lambda x : np.random.choice(x, 30).mean(), 1, pool)

print(res.shape)
print("Time used: %f s" % (time.time() - start))

(10000,)
Time used: 0.694955 s


### To be noted that this is not a good example to show the performance of NumPy, which is good at matrix/vector calculation.
---------

In [25]:
bList = list(range(10000))

In [26]:
def square_list(seq):
    res = []
    for i in seq:
        res.append(i * i)
    return res

%timeit(square_list(bList))

1000 loops, best of 3: 1.48 ms per loop


In [27]:
%timeit([val * val for val in bList])

1000 loops, best of 3: 836 µs per loop


In [28]:
arr = np.array(bList)

%timeit(arr * arr)

The slowest run took 18.43 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 10.6 µs per loop


In [29]:
x = np.array([1,2,3])

print(x*x)

[1 4 9]


### Numba Module

In [30]:
import numba

In [31]:
def fib(n):
    a, b = 0, 1
    for i in range(n):
        a, b = b, a + b
    return a

%timeit(fib(50))

100000 loops, best of 3: 4.79 µs per loop


In [32]:
@numba.jit
def fib(n):
    a, b = 0, 1
    for i in range(n):
        a, b = b, a + b
    return a

%timeit(fib(50))

The slowest run took 377172.48 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 232 ns per loop


In [33]:
%load_ext Cython

In [34]:
%%cython

def fib(n):
    a, b = 0, 1
    for i in range(n):
        a, b = b, a + b
    return a

In [35]:
%timeit(fib(50))

100000 loops, best of 3: 2.32 µs per loop
